In [16]:
from math import exp
import numpy as np
from sigmoid import Sigmoid
from softmax import Softmax
from data import shuffle_generator
from sklearn.datasets import load_digits
from cce_loss_function2 import Categorical_Cross_Entropy_Loss

#data
digits = load_digits()
x = digits.data
y = digits.target

#inputs
x = x/np.max(x)
x = np.float32(x)

#target
y = np.eye(10, dtype=int)[y]

#delete last 3 rows
x = x[:-3]
y = y[:-3]

# Parameters
batch_size = 13
shuffled_data_generator = shuffle_generator(x, y, batch_size)
input_size = 64
output_size = 10



class MLP_layer():
    def __init__(self, num_inputs, num_units, activation_function, use_bias=True):
        self.num_inputs = num_inputs
        self.num_units = num_units
        self.use_bias = use_bias
        self.weights = np.random.normal(loc=0.0, scale=0.2, size=(self.num_inputs, self.num_units ))
        self.bias = np.zeros((self.num_units,))
        if activation_function == "sigmoid":
            self.act_function = Sigmoid()
        elif activation_function == "softmax":
            self.act_function = Softmax()

    # adjust weights
    def set_weights(self, weights, biases):
        self.weights = weights
        self.bias = biases

    def forward(self, x):
        if x.shape != (batch_size, self.num_inputs):
            raise AssertionError(f"The input should be of shape ({batch_size}, {self.num_inputs}) but you idiot gave me {x.shape}!?")
        #pre_activations = self.weights @ x + np.transpose(self.bias)
        pre_activations = np.dot(x, self.weights) + np.transpose(self.bias)
        activations = self.act_function(pre_activations)
        
        return activations
    
    def weights_backward(self, d_preacts, preacts):
        d_weights = self.act_function.backwards(preacts)*np.sum(self.weights, axis=0)*d_preacts
        d_inputs = d_preacts
        return d_weights, d_inputs
    
    def backward(self, error_signal, inputs):
        dL_dW = np.dot(inputs.T, error_signal)
        dL_dinput = np.dot(error_signal, self.weights.T)
        
        return dL_dW, dL_dinput

class MLP():
    def __init__(self, num_layers, num_units_for_layers, activation_functions):
        if num_layers != len(num_units_for_layers)-1:
            raise AssertionError(f"You have to specify as many num_units as you got layers!!")
        if set(activation_functions) != {"sigmoid", 'softmax'} and len(activation_functions) == num_layers:
            raise AssertionError(f"You have to specify as many activation_functions as you got layers and only use 'softmax' or 'sigmoid'")
        self.layer_list = [MLP_layer(num_units_for_layers[l], num_units_for_layers[l+1], activation_functions[l]) for l in range(num_layers)]
        
    def forward(self, x, target):
        for i in range(len(self.layer_list)):
            x = self.layer_list[i].forward(x)
            
        func = Categorical_Cross_Entropy_Loss()
        loss = func(x,target)
        
        print(loss)
        return loss
    
    def get_data_forward(self, data_generator):
        
        batches = []
        
        while True:
            try:
                batches.append(next(shuffled_data_generator))
                
            except StopIteration:
                break
                
        for i in range(len(batches)):
            self.forward(batches[i][0], batches[i][1])
            
    def backward(self, x, y):
        
        layer_info = [{} for _ in range(len(self.layer_list))]
    
    
#create MLP
my_cute_mlp = MLP(2, [input_size,16,output_size], ["sigmoid", "softmax"])

#use MLP forward
my_cute_mlp.get_data_forward(shuffled_data_generator)




[2.28909737 2.33181507 2.25191245 2.35078471 2.30317119 2.29098696
 2.34430418 2.31284058 2.3673687  2.33093638 2.32889802 2.25313236
 2.32483727]
[2.31292752 2.32887184 2.25403283 2.26185213 2.3029145  2.15869798
 2.1671388  2.28910752 2.33051945 2.17083509 2.37261183 2.34225013
 2.28717372]
[2.3314994  2.16995038 2.34953481 2.3166455  2.37227609 2.24564902
 2.32893631 2.30344134 2.34529219 2.30721197 2.30198422 2.3168385
 2.25456598]
[2.17500314 2.30970037 2.32308729 2.3113921  2.32017878 2.15864508
 2.32839801 2.30207612 2.37440627 2.37397212 2.37350485 2.32628977
 2.35255171]
[2.3278907  2.31848351 2.3161014  2.28852568 2.34410137 2.32480122
 2.32602582 2.33380927 2.37068802 2.32155409 2.31861857 2.1895738
 2.31547548]
[2.33617149 2.16077781 2.35090459 2.31412876 2.31946323 2.32828963
 2.31219897 2.30447924 2.31484733 2.32395877 2.31098367 2.36513621
 2.3108514 ]
[2.30053091 2.33036107 2.25302723 2.26447314 2.30562755 2.31257319
 2.30480343 2.31140233 2.3246194  2.16405458 2.292096